In [0]:
#!python -m spacy download en_core_web_lg
#!pip3 install --upgrade --force-reinstall tensorflow-gpu==1.9.0 --user
#!pip3 install --upgrade tensorflow
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords # corpus implies collection of words of same type
from nltk.stem.porter import PorterStemmer

#nlp= spacy.load('en_core_web_lg')
#nlp.Defaults.stop_words.add('s')
#nlp.vocab['s'].is_stop=True

#nlp.max_length = 5198623
dataset = pd.read_excel('Data_Train.xlsx',delimiter='\t',quoting=3)
testing_data = pd.read_excel('Data_Test.xlsx',delimiter='\t',quoting=3)
#quoting = 3 for ignoring the double quotes



corpus = []

for i in range(0,7628):
    review = re.sub('[^a-zA-Z]',' ',dataset['STORY'][i])
    # re.sub function with ^ only keeps character and removes all the punctuation
    #' '= space so that two words dont joint
    review = review.lower()
    # lowercase character
    review = review.split()
    # to convert string into list
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    # set for faster execution
    # checking irrelevant words in stopwords e.g this
    # PorterStemmer converts 'loved' into 'love' into their original stem

    review = ' '.join(review) 
    # convert the list into string again. # ' ' - so that words dont join
    corpus.append(review) # to append the review to the corpus list

In [0]:
corpus1 = []
for i in range(0,2748):
    review1 = re.sub('[^a-zA-Z]',' ',testing_data['STORY'][i])
    review1 = review1.lower()
    review1 = review1.split()
    ps1 = PorterStemmer()
    review1 = [ps1.stem(word1) for word1 in review1 if not word1 in set(stopwords.words('english'))]
    review1 = ' '.join(review1) 
    corpus1.append(review1) # to append the review to the corpus list
     

In [0]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [0]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))
                           

In [0]:
corpus = np.array(corpus)
corpus1 = np.array(corpus1)
list_train = [corpus[i:i+100] for i in range(0,corpus.shape[0],100)]
list_test = [corpus1[i:i+100] for i in range(0,corpus1.shape[0],100)]

In [0]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x) for x in list_train]
elmo_test = [elmo_vectors(x) for x in list_test]

In [0]:
# save elmo_train_new
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [0]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [0]:
import pickle
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [0]:
dataset = pd.read_excel('Data_Train.xlsx',delimiter='\t',quoting=3)
y = dataset.iloc[:,1].values


In [6]:

from sklearn.ensemble import RandomForestClassifier
classifier1 = RandomForestClassifier(n_estimators = 300, criterion = 'entropy', random_state = 0)
classifier1.fit(elmo_train_new, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [7]:
y_pred = classifier1.predict(elmo_test_new)

submission = pd.DataFrame({'SECTION':y_pred})
filename = 'prediction_news.xlsx'
submission.to_excel(filename,index=False)
print('Saved file: ' + filename)

Saved file: prediction_news.xlsx


In [0]:
#!pip install --upgrade tensorflow_gpu==1.5.0
#!nvcc --version
#!apt-get install cuda-libraries-9-0